# Answering Business Questions using SQL

### I'll be working with the Chinook database which containes infromation about music tracks sales. <br> Let's first connect to the database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

### Let's start by getting familiar with our data.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

In [3]:
%%sql
SELECT *
FROM customer
LIMIT 3;

In [4]:
%%sql
SELECT *
FROM invoice
LIMIT 3;

>The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

>Artist Name	Genre<br>
>Regal	Hip-Hop<br>
>Red Tone	Punk<br>
>Meteor and the Girls	Pop<br>
>Slim Jim Bites	Blues<br>

>The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

>I'll need to find out which genres sell the most tracks in the USA, and make a recommendation for the three artists whose albums we should purchase for the store.

In [5]:
%%sql
WITH usa_tracks_sold AS
   (
    SELECT il.* 
    FROM invoice_line AS il
    INNER JOIN invoice AS i on il.invoice_id = i.invoice_id
    INNER JOIN customer AS c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )
SELECT
    g.name AS genre,
    COUNT(uts.invoice_line_id) AS tracks_sold,
    ROUND(CAST(COUNT(uts.invoice_line_id) AS FLOAT) / 
    (SELECT COUNT(*) from usa_tracks_sold),4)*100 AS percentage_sold
FROM usa_tracks_sold AS uts
INNER JOIN track AS t on t.track_id = uts.track_id
INNER JOIN genre AS g on g.genre_id = t.genre_id
GROUP BY genre
ORDER BY tracks_sold DESC
LIMIT 15;

### As can be seen the majority of the sales belong to the rock gnere. <br> We can extend the query to include the the artist names which are selling the most tracks.<br> This would give us the option to to further our information resolution in order to choose new albums to sell.

In [6]:
%%sql
WITH artist_sales AS 
    (SELECT
         il.invoice_line_id AS invoice_line_id,
         ar.name AS artist_name,
         t.genre_id AS genre_id
    FROM invoice_line AS il
    INNER JOIN invoice AS i on il.invoice_id = i.invoice_id
    INNER JOIN customer AS c on i.customer_id = c.customer_id
    INNER JOIN track AS t ON il.track_id = t.track_id
    INNER JOIN album AS al ON t.album_id = al.album_id
    INNER JOIN artist AS ar ON al.artist_id = ar.artist_id
    WHERE c.country = "USA"
    )
SELECT 
    ars.artist_name AS artist_name,
    g.name AS genre_name,
    COUNT(ars.invoice_line_id) AS tracks_sold,
    ROUND(CAST(COUNT(ars.invoice_line_id) AS FLOAT) / 
    (SELECT COUNT(*) FROM artist_sales),4)*100 AS percentage_sold
FROM artist_sales AS ars
INNER JOIN genre AS g ON ars.genre_id = g.genre_id
WHERE genre_name IN ('Hip Hop/Rap','Alternative & Punk','Blues','Pop')
GROUP BY artist_name, genre_name
ORDER BY tracks_sold DESC 
LIMIT 50;   


### Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

>Red Tone (Punk)<br>
>Slim Jim Bites (Blues)<br>
>Meteor and the Girls (Pop)<br>

### Even tough an hip op album is ranked at number 4 at the track sales, aggregating the pop genre produces more sales.

### Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase
### The task is to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others

In [7]:
%%sql
SELECT * 
FROM employee
WHERE title = 'Sales Support Agent';

In [8]:
%%sql
WITH support_rep_sales AS
    (SELECT
         c.support_rep_id AS support_rep_id,
         e.first_name AS first_name,
         e.last_name AS last_name,
         c.country AS country,
         i.total AS total
     FROM employee AS e
     INNER JOIN customer AS c ON e.employee_id = c.support_rep_id
     INNER JOIN invoice AS i ON c.customer_id = i.customer_id
     WHERE title = 'Sales Support Agent' 
    )
SELECT 
    support_rep_id,
    first_name || " " || last_name AS full_name,
    country,
    ROUND(SUM(total),2) AS total,
    ROUND(SUM(Total) / (SELECT SUM(total) FROM support_rep_sales) * 100,2) AS percentage_sold
FROM support_rep_sales
GROUP BY 3
ORDER BY 4 DESC;

 * sqlite:///chinook.db
Done.


support_rep_id,full_name,country,total,percentage_sold
3,Jane Peacock,USA,1040.49,22.09
3,Jane Peacock,Canada,535.59,11.37
3,Jane Peacock,Brazil,427.68,9.08
4,Margaret Park,France,389.07,8.26
3,Jane Peacock,Germany,334.62,7.11
4,Margaret Park,Czech Republic,273.24,5.8
3,Jane Peacock,United Kingdom,245.52,5.21
4,Margaret Park,Portugal,185.13,3.93
3,Jane Peacock,India,183.15,3.89
3,Jane Peacock,Ireland,114.84,2.44


### Based on the data Steve Johnson's sales preformance is not on par with Jane Peacock's and Margaret Park's sales preformance. It's worth to consider giving Steve additonal marketing and sales training in order to improve his abilities.